### Annotation Selection of the hg38 genome

##### Before you begin, please make sure you have the proper files downloaded

In [11]:
#Download link for gencode annotation file (gff3): https://www.gencodegenes.org/human/release_38.html
##Make sure to download the "comprehensive gene annotation file (CHR)" in the gff3 format

#Download link for hg38 archetype motif file: https://www.vierstra.org/resources/motif_clustering
##Download the hg38 archetype scan file alongside its tabix file

In [12]:
import pandas as pd
import numpy as np
import gffutils
import pysam

##### If this is your first time importing the gff3 database, run this command

In [13]:
#This can take a long time, so only run this once (remove the #)

#db = gffutils.create_db('gencode.v38.annotation.gff3', dbfn='gff3db.db', force=True, keep_order=True, 
#merge_strategy='create_unique', sort_attribute_values=True) 

#replace gff3 with your gff3 file, dbfn can be whichever name you want to store the transposed file as


##### Feature the selected database 

In [14]:
db = gffutils.FeatureDB('gff3db.db', keep_order=True)

##### Select the desired range

In [15]:
chr = 'chr1' #Replace with wanted chromosome location
start = 207320867 #Replace with start coordinate
stop = 207320897 #Replace with end coordinate
slct = db.region(region=(chr, start, stop), completely_within=False)


##### Iterate through the region and transpose it to a dictionary

In [16]:
records = []
for gene in slct:
    records.append({
        "seqid": gene.seqid,
        "source": gene.source,
        "featuretype": gene.featuretype,
        "start": gene.start,
        "end": gene.end,
        "strand": gene.strand,
        "score": gene.score,
        "attributes": gene.attributes  # Attributes as a dictionary (will be expanded later)
    })

##### Next, lets integrate the hg38 regulatory elements into this database:

In [17]:
input_file = 'hg38.archetype_motifs.v1.0.bed.gz' #Replace with the RE file
tabix = pysam.TabixFile(input_file)
source_name = 'atlas' #replace with reference cell type
#It may say that the index file is older than the data file, you can ignore this for now

In [18]:
#Create dictionary using the input selection
for row in tabix.fetch(chr,start,stop):
    assets = row.split()
    records.append({
        "seqid": assets.pop(0),
        "source": source_name,
        "start": int(assets.pop(0)),
        "end": int(assets.pop(0)),
        "featuretype": 'RE',
        "strand": assets.pop(2),
        "attributes": ({"ID": assets.pop(2), "matchscore": float(assets.pop(1)), "cluster":assets.pop(0)})  # Attributes as a dictionary (will be expanded later)
    })

##### Expand out the attributes and format the dictionary as a database

In [21]:
df = pd.DataFrame(records)
attributes_df = df["attributes"].apply(pd.Series)
df = pd.concat([df.drop(columns=["attributes"]), attributes_df], axis=1)

display(df)

,seqid,source,start,end,featuretype,strand,ID,matchscore,cluster
0,chr1,atlas,207320851,207320868,RE,-,SALL4_HUMAN.H11MO.0.B,9.9542,KLF/SP/2
1,chr1,atlas,207320852,207320869,RE,-,EGR2_HUMAN.H11MO.0.A,11.5015,KLF/SP/2
2,chr1,atlas,207320853,207320873,RE,-,KLF15_HUMAN.H11MO.0.A,11.5516,GC-tract
3,chr1,atlas,207320854,207320872,RE,+,TBX3_MOUSE.H11MO.0.B,7.7390,TBX/3
4,chr1,atlas,207320855,207320872,RE,-,KLF1_MOUSE.H11MO.0.A,10.1096,KLF/SP/2
...,...,...,...,...,...,...,...,...,...
62,chr1,atlas,207320893,207320910,RE,-,EGR2_HUMAN.H11MO.0.A,7.8496,KLF/SP/2
63,chr1,atlas,207320894,207320907,RE,+,ZSCAN4_C2H2_1,3.8977,ZSCAN4
64,chr1,atlas,207320895,207320906,RE,+,KLF9_MA1107.1,8.4085,KLF/SP/1
65,chr1,atlas,207320895,207320912,RE,-,EGR2_HUMAN.H11MO.0.A,7.8496,KLF/SP/2


##### Expand out the single list attributes as strings

In [22]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)

display(df)

,seqid,source,start,end,featuretype,strand,ID,matchscore,cluster
0,chr1,atlas,207320851,207320868,RE,-,SALL4_HUMAN.H11MO.0.B,9.9542,KLF/SP/2
1,chr1,atlas,207320852,207320869,RE,-,EGR2_HUMAN.H11MO.0.A,11.5015,KLF/SP/2
2,chr1,atlas,207320853,207320873,RE,-,KLF15_HUMAN.H11MO.0.A,11.5516,GC-tract
3,chr1,atlas,207320854,207320872,RE,+,TBX3_MOUSE.H11MO.0.B,7.7390,TBX/3
4,chr1,atlas,207320855,207320872,RE,-,KLF1_MOUSE.H11MO.0.A,10.1096,KLF/SP/2
...,...,...,...,...,...,...,...,...,...
62,chr1,atlas,207320893,207320910,RE,-,EGR2_HUMAN.H11MO.0.A,7.8496,KLF/SP/2
63,chr1,atlas,207320894,207320907,RE,+,ZSCAN4_C2H2_1,3.8977,ZSCAN4
64,chr1,atlas,207320895,207320906,RE,+,KLF9_MA1107.1,8.4085,KLF/SP/1
65,chr1,atlas,207320895,207320912,RE,-,EGR2_HUMAN.H11MO.0.A,7.8496,KLF/SP/2


##### Use the following code to find an RE from the human genome browser

In [23]:
lst_of_id = df[df["ID"] == 'ZFX_MOUSE.H11MO.0.B'] ##Replace with the RE ID
lst_of_id.dropna(how='all', axis=1, inplace=True)
lst_of_id

/tmp/ipykernel_235/981348799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lst_of_id.dropna(how='all', axis=1, inplace=True)


,seqid,source,start,end,featuretype,strand,ID,matchscore,cluster
26,chr1,atlas,207320872,207320883,RE,+,ZFX_MOUSE.H11MO.0.B,7.7766,ZFX


In [24]:
lst_of_id[lst_of_id['matchscore'] == 7.7766] ###Replace with matchscore

,seqid,source,start,end,featuretype,strand,ID,matchscore,cluster
26,chr1,atlas,207320872,207320883,RE,+,ZFX_MOUSE.H11MO.0.B,7.7766,ZFX
